In [ ]:
file_path = "C:/Users/loype/OneDrive - Singapore University of Technology and Design/Term 7/Capstone/ST-Engineering-1H2024-Results-Financial-Statements.pdf"

In [7]:

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [21]:
print(len(pages))
print(pages[1].page_content)
print(pages[1].metadata)

27
 
 
 
 
 
Page 1 of 26 
 
                                                               SINGAPORE TECHNOLOGIES ENGINEERING LTD  
UNAUDITED RESULTS FOR THE FIRST HALF -YEAR  ENDED 30 JUNE 202 4 
 
1 CONSOLIDATED INCOME STATEMENT FOR THE FIRST HALF -YEAR  ENDED 30 JUNE 202 4 
 
 
NM: Not Meaningful  
 
Note :   
   Amounts may not add to totals shown d ue to rounding  
  
1H2024 1H2023 +/(-)
$'000 $'000 %
Revenue 5,520,394   4,863,416   13.5
Cost of sales (4,467,499)  (3,883,004)  15.1
Gross profit 1,052,895   980,412      7.4
Distribution and selling expenses (refer to para 1.6) (104,584)     (103,469)     1.1
Administrative expenses (refer to para 1.7) (385,900)     (359,368)     7.4
Other operating expenses (refer to para 1.8) (86,238)       (83,504)       3.3
Other income/ (expenses), net (refer to para 1.9) 8,382           (13,250)       NM
Profit from operations 484,555      420,821      15.1
Non-operating expenses, net (59)               (202)             (70.8)
Share of resul